In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import joblib

# 1. Load and Prepare MULTIVARIATE Data ## MULTIVARIATE CHANGE ##
# Load the combined dataset with weather features
df_multi = pd.read_csv('combined_dataset.csv', index_col='datetime', parse_dates=True)

# Ensure 'stage_m' is the first column, as our sequence function will use it as the target
df_multi = df_multi[['stage_m'] + [col for col in df_multi.columns if col != 'stage_m']]

print("Columns in order:", df_multi.columns.tolist())

# 2. Split Data with a Gap (following paper methodology, as in context7)
# Dataset: 1995-10-01 to 2025-06-01 (~29.67 years)
# Paper approach: 76% training, 23% validation with gap
# Training: 76% of 29.67 years ≈ 22.55 years → 1995-10-01 to 2018-04-21
# Gap: ~8 months (2018-04-21 to 2019-01-01)
# Validation: 2019-01-01 to 2025-06-01
split_date_train_end = '2018-04-21'
split_date_val_start = '2019-01-01'
train_data = df_multi.loc[df_multi.index < split_date_train_end]
val_data = df_multi.loc[df_multi.index >= split_date_val_start]

print(f"Training period: {train_data.index.min()} to {train_data.index.max()}")
print(f"Validation period: {val_data.index.min()} to {val_data.index.max()}")
print(f"Training samples: {len(train_data)} ({len(train_data)/(len(train_data)+len(val_data))*100:.1f}%)")
print(f"Validation samples: {len(val_data)} ({len(val_data)/(len(train_data)+len(val_data))*100:.1f}%)")
print(f"Gap between training and validation: {split_date_val_start} - {split_date_train_end}")

# 3. Scale Data ## MULTIVARIATE CHANGE ##
# The scaler will now be fit on all features
scaler = StandardScaler()
scaled_train_data = scaler.fit_transform(train_data)
scaled_val_data = scaler.transform(val_data)

# Save the fitted scaler using joblib
joblib.dump(scaler, 'multivariate_combined_scaler.joblib')
print("Scaler saved to 'multivariate_combined_scaler.joblib'")

# 4. Create Sequences ## MULTIVARIATE CHANGE ##
# The function now takes all columns for X, but still only the first column for y
N_PAST = 7    # Use 7 days of history
N_FUTURE = 7  # Predict 7 days into the future

def create_sequences(data, n_past, n_future):
    X, y = [], []
    target_col_index = 0 # 'stage_m' is at index 0
    for i in range(n_past, len(data) - n_future + 1):
        # Input X: takes n_past days of ALL features
        X.append(data[i - n_past:i, :])
        # Output y: takes n_future days of ONLY the target feature
        y.append(data[i:i + n_future, target_col_index])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(scaled_train_data, N_PAST, N_FUTURE)
X_val, y_val = create_sequences(scaled_val_data, N_PAST, N_FUTURE)
# Note: No .reshape() is needed for X, as it's already 3D

print(f"\nNew sequence shapes:")
print(f"X_train shape: {X_train.shape}") # Should be (samples, 7, num_features)
print(f"y_train shape: {y_train.shape}")   # Should be (samples, 7)

# 5. Build Stacked LSTM Encoder-Decoder (Same architecture, just n_features will be > 1)
n_features = X_train.shape[2]
print(f"Number of features being used: {n_features}")
UNITS = 128
DROPOUT_RATE = 0.15

inputs = Input(shape=(N_PAST, n_features))
encoder_l1 = LSTM(UNITS, return_sequences=True, dropout=DROPOUT_RATE)(inputs)
encoder_l2, state_h, state_c = LSTM(UNITS, return_sequences=False, dropout=DROPOUT_RATE, return_state=True)(encoder_l1)
encoder_states = [state_h, state_c]
decoder_inputs = RepeatVector(N_FUTURE)(encoder_l2)
decoder_l1 = LSTM(UNITS, return_sequences=True, dropout=DROPOUT_RATE)(decoder_inputs, initial_state=encoder_states)
decoder_l2 = LSTM(UNITS, return_sequences=True, dropout=DROPOUT_RATE)(decoder_l1)
output = TimeDistributed(Dense(1))(decoder_l2)
model = Model(inputs=inputs, outputs=output)

# 6. Compile Model (Same as our best univariate model)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer, loss='mean_squared_error')
model.summary()

# 7. Train the Model ## MULTIVARIATE CHANGE ##
# Using a new filename for the saved model
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(filepath='best_multivariate_combined_model.keras', save_best_only=True, monitor='val_loss', mode='min')

history = model.fit(
    X_train, y_train,
    epochs=150,
    batch_size=128,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

# 8. Evaluation and Visualization ## MULTIVARIATE CHANGE ##
# The logic for inverse scaling the predictions is now more complex

# Load the scaler back (this demonstrates how you'd use it in production)
loaded_scaler = joblib.load('multivariate_combined_scaler.joblib')

best_model = tf.keras.models.load_model('best_multivariate_combined_model.keras')
predictions_scaled = best_model.predict(X_val).squeeze()

# --- Special handling for inverse scaling ---
# Create a dummy array shaped like the original data the scaler was fit on
# We will overwrite the first column (our target) with our predictions
dummy_preds = np.zeros((len(predictions_scaled.flatten()), n_features))
dummy_preds[:, 0] = predictions_scaled.flatten()
# Inverse transform the entire dummy array
rescaled_preds = loaded_scaler.inverse_transform(dummy_preds)
# Extract just the first column (our rescaled predictions) and reshape
predictions_original = rescaled_preds[:, 0].reshape(y_val.shape)

# Do the same for the true values
dummy_true = np.zeros((len(y_val.flatten()), n_features))
dummy_true[:, 0] = y_val.flatten()
rescaled_true = loaded_scaler.inverse_transform(dummy_true)
y_true_original = rescaled_true[:, 0].reshape(y_val.shape)
# --- End of special handling ---

print("\n--- MULTIVARIATE Model Performance on Validation Set ---")
for i in range(N_FUTURE):
    day = i + 1
    mae = mean_absolute_error(y_true_original[:, i], predictions_original[:, i])
    rmse = np.sqrt(mean_squared_error(y_true_original[:, i], predictions_original[:, i]))
    r2 = r2_score(y_true_original[:, i], predictions_original[:, i])
    print(f"Day {day} Ahead -> MAE: {mae:.4f} m, RMSE: {rmse:.4f} m, R²: {r2:.4f}")

# Plotting code remains the same
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.title('Multivariate Model Learning Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
plt.show()

val_dates = val_data.index[N_PAST:len(y_true_original) + N_PAST]
plt.figure(figsize=(15, 8))
plt.plot(val_dates, y_true_original[:, 0], label='Actual Values', color='blue', alpha=0.8)
plt.plot(val_dates, predictions_original[:, 0], label='Day 1 Forecast', color='red', linestyle='--', alpha=0.9)
plt.plot(val_dates, predictions_original[:, 2], label='Day 3 Forecast', color='orange', linestyle=':', alpha=0.8)
plt.plot(val_dates, predictions_original[:, 6], label='Day 7 Forecast', color='green', linestyle='-.', alpha=0.8)
plt.title('Multivariate 7-Day Water Level Forecast vs. Actual')
plt.xlabel('Date')
plt.ylabel('Water Level (m)')
plt.legend()
plt.grid(True)
plt.show()

2025-06-19 18:28:16.349543: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-19 18:28:16.431376: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750346896.480050  115610 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750346896.492039  115610 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750346896.595740  115610 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Columns in order: ['stage_m', 'discharge_cms', 'tavg', 'prcp', 'wspd', 'pres', 'rhum']
Training period: 1995-10-01 00:00:00 to 2018-04-20 00:00:00
Validation period: 2019-01-01 00:00:00 to 2025-06-01 00:00:00
Training samples: 8216 (77.8%)
Validation samples: 2344 (22.2%)
Gap between training and validation: 2019-01-01 - 2018-04-21
Scaler saved to 'multivariate_combined_scaler.joblib'

New sequence shapes:
X_train shape: (8203, 7, 7)
y_train shape: (8203, 7)
Number of features being used: 7


I0000 00:00:1750346901.380228  115610 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 7, 7)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 7, 128)    │     69,632 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 128),     │    131,584 │ lstm[0][0]        │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 7, 128)    │          0 │ lstm_1[0][0]      │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 7, 128)    │    131,584 │ repeat_vector[0]… │
│                     │                   │            │ lstm_1[0][1],     │
│                     │                   │            │ lstm_1[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 7, 128)    │    131,584 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 7, 1)      │        129 │ lstm_3[0][0]      │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

...

 Trainable params: 464,513 (1.77 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150


I0000 00:00:1750346910.214337  115723 cuda_dnn.cc:529] Loaded cuDNN version 90300


65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - loss: 0.6786 - val_loss: 0.6319
Epoch 2/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.5142 - val_loss: 0.5777
Epoch 3/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.4792 - val_loss: 0.5731
Epoch 4/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.4631 - val_loss: 0.5580
Epoch 5/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.4784 - val_loss: 0.5505
Epoch 6/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.4823 - val_loss: 0.5525
Epoch 7/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.4420 - val_loss: 0.5487
Epoch 8/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.4464 - val_loss: 0.5609
Epoch 9/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.4385 - val_loss: 0.5525
Epoch 10/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.4244 - val_loss: 0.5447
Epoch 11/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.4482 - val_loss: 0.5409
Epoch 12/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.4